In [1]:
import pandas as pd

### load the data

In [2]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
df_target = pd.read_csv('sample_submission.csv')

In [15]:
df_train.isna().sum()


id             0
keyword        0
location    2533
text           0
target         0
dtype: int64

In [16]:
df_test.isna().sum()


id             0
keyword        0
location    1105
text           0
dtype: int64

In [5]:
df_target.isna().sum()


id        0
target    0
dtype: int64

In [13]:
df_train.keyword = df_train.keyword.fillna(df_train.keyword.mode()[0])
df_test.keyword = df_test.keyword.fillna(df_test.keyword.mode()[0])


In [14]:

df_train

,id,keyword,location,text,target
0,1,fatalities,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,fatalities,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,fatalities,NaN,All residents asked to 'shelter in place' are ...,1
3,6,fatalities,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,fatalities,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,fatalities,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,fatalities,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,fatalities,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,fatalities,NaN,Police investigating after an e-bike collided ...,1


,id,target
0,0,0
1,2,0
2,3,0
3,9,0
4,11,0
...,...,...
3258,10861,0
3259,10865,0
3260,10868,0
3261,10874,0
